### Collab filtering/MF model V1.1

* try intermediate-simple collaborative filtering/MF/implicit recc embedding model
* create new "id" - semi distinct tuple of distinguishing variables
    * Could consider adding previous city/country visited to tuple - would reduce data but greatly improve this model. For that to make sense, we would also need to do a shared embedding so that city id lag1 and city id will have the same embedding weights learned. 
    * Could also add discretized number of locations in a trip `total_rows` (1-3,4-5,6+?) as feature , +- step/stage in a trip - https://pbpython.com/natural-breaks.html
* Start with stupid approach: ~9k most popular items. highly unbalanced. Dot product .
    * Alt simple baseline - SVD (but that loses on embeddings per variable. Instead we'd use just the id)
    
    
    
 - Possible negatives sampling code : 
     * https://curiousily.com/posts/build-a-recommender-system-using-keras-and-tensorflow2-in-python/
     * tf.datasets - https://stackoverflow.com/questions/58520594/tf-data-dataset-on-each-epoch-only-train-with-a-sub-sample-of-the-full-datase
     
     
     
 * ALT - get all combinations (itertools) - https://stackoverflow.com/questions/43800390/how-to-create-all-combinations-column-wise-for-multiple-variables-in-pandas
 
 * Get all combinations: `pd.crosstab(df["city_id"],df["ID"]).stack().reset_index()`
 * Keras + sparse generator - https://stackoverflow.com/questions/41538692/using-sparse-matrices-with-keras-and-tensorflow
     * https://www.kaggle.com/luisgarcia/keras-nn-with-parallelized-batch-training
 
 ###### V1.1: Add lag feature - hotel_country
 * +- first hotel country, and lag1 hotel country ?
 
 * Would be interesting to combine with "next country" prediction task

* https://www.bookingchallenge.com/

* Predict `city_id`
        * Metric: P@4

##### Dataset
The training dataset consists of over a million of anonymized hotel reservations, based on real data, with the following features:
*    user_id - User ID
*    check-in - Reservation check-in date
*    checkout - Reservation check-out date
*    affiliate_id - An anonymized ID of affiliate channels where the booker came from (e.g. direct, some third party referrals, paid search engine, etc.)
*    device_class - desktop/mobile
*    booker_country - Country from which the reservation was made (anonymized)
*    hotel_country - Country of the hotel (anonymized)
*    city_id - city_id of the hotel’s city (anonymized)
*    utrip_id - Unique identification of user’s trip (a group of multi-destinations bookings within the same trip)


* Each reservation is a part of a customer’s trip (identified by utrip_id) which includes at least 4 consecutive reservations. The check-out date of a reservation is the check-in date of the following reservation in their trip.

* The evaluation dataset is constructed similarly, however the city_id of the final reservation of each trip is concealed and requires a prediction.

 
###### Evaluation criteria
The goal of the challenge is to predict (and recommend) the final city (city_id) of each trip (utrip_id). We will evaluate the quality of the predictions based on the top four recommended cities for each trip by using Precision@4 metric (4 representing the four suggestion slots at Booking.com website). When the true city is one of the top 4 suggestions (regardless of the order), it is considered correct.

------------------------------------------------------

* If we are given  the country in question, then this problem is maybe more of a _learning to rank_ problem. (Rather than massively multiclass). 
    * CatBoost learning to rank on ms dataset (0/1):  https://colab.research.google.com/github/catboost/tutorials/blob/master/ranking/ranking_tutorial.ipynb
        * https://catboost.ai/docs/concepts/loss-functions-ranking.html
        * for CB ranking,  all objects in dataset must be grouped by group_id - this would be user/trip id X country, in our case. (Still need to add negatives, within each such subgroup/group/"query"). 

    * lightFM - ranking (implicit interactions)
        * https://github.com/qqwjq/lightFM

    * lstm/w2v - next item recomendation
    * dot product between different factors as features (recc.)
    * xgboost ap - https://www.kaggle.com/anokas/xgboost-2
* Relevant: Kaggle expedia hotel prediction: https://www.kaggle.com/c/expedia-hotel-recommendations/discussion  

* ALSO: `implicit interaction` - reccommendation problem (We have only positive feedback, no ranked/negative explicit feedback)'


* __BASELINE__ to beat: 4 most popular by country ; 4 most popular by affiliate_id X booker_country X hotel_country (X month?)
    * Ignore/auto answer the 4 most popular for countries with less than 4 unique cities in data
 
 
* Likely approach : build a model (and targets/negatives) per country.

-----------
#### Data notes:
* Long tail of cities and countries
* Some (31%) countries have 4 or less unique cities - for those return fixed answer/prediction ?  -
    * CAN'T! In test set, we will not have the country ID :(
    
    
----------------------
MF - embedding model

* https://blog.tensorflow.org/2020/09/introducing-tensorflow-recommenders.html
* Implicit recommendations - needs negs
    * example of explicit (simple): https://petamind.com/build-a-simple-recommender-system-with-matrix-factorization/
* sample negatives - how ? TFRS requires tf.dataset overhead (And confuses me with what user id should be )
    * https://www.kaggle.com/skihikingkevin/some-recommender-system-implementations
    
    
Simple keras example of multiple inputs : 
* keras topologies
* https://stackoverflow.com/questions/61722973/why-keras-embedding-not-learning-for-recommendation-system


*Tensorflow ranking (seems in beta) : https://colab.research.google.com/github/tensorflow/ranking/blob/master/tensorflow_ranking/examples/handling_sparse_features.ipynb#scrollTo=HfDMGnZY9eVO


Negative pairs training with generator - https://towardsdatascience.com/building-a-recommendation-system-using-neural-network-embeddings-1ef92e5c80c9
* See code. 

In [1]:
# Recommenders embedding - fit generator
# https://towardsdatascience.com/building-a-recommendation-system-using-neural-network-embeddings-1ef92e5c80c9
# Also has code for generator to generate positive, negative pairs per batch - good for siamese/triplets/metric! 

import numpy as np
import random
np.random.seed(100)
random.seed(100)

Possible approahc + negatives - https://github.com/zhangruiskyline/DeepLearning/blob/master/doc/Recommendation.md#ranking 


* Negative sampling from the sparse user-item cooccurrence matrix
    * https://stackoverflow.com/questions/49971318/how-to-generate-negative-samples-in-tensorflow


In [2]:
# from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
import numpy as np

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, GroupShuffleSplit

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import TopKCategoricalAccuracy, Precision, SparseTopKCategoricalAccuracy # @4
from tensorflow.keras.utils import to_categorical
from tensorflow import feature_column
from tensorflow.keras import layers

%matplotlib inline

pd.set_option("display.max_columns", 90)

In [3]:
# ## https://www.tensorflow.org/guide/mixed_precision ## TF mixed precision - pytorch requires other setup
# from tensorflow.keras.mixed_precision import experimental as mixed_precision

# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)
# ## will need to correct in places, e.g.: 
# ## outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)

#### Features to add:
* Lag 
* Rank (popularity) of city, country (in general, +- given booker country)
* Count of hotel; user, trip size ? (may be leaky )
* Seasonal features - Holidays? , datetime

Aggregate feats:
* user changed country? last booking (lag 1) country change? 
* max/min/avg popularity rank of previous locations visited



We should create a dictionary of the rank, count, city/country etc' feats, so we can easily merge them when making more "negative" samples/feats for ranking.


* Consider using a df2 of df without dates + drop_duplicates, +- without user/trip id (After calcing that) .


Leaky or potentially leaky (Dependso n test set): 
* Target freq features - frequency of target city, given source county +- affiliate +- month of year +- given country (and interactions of target freq). 
    * Risk of leaks - depends of test data has temporal split or not. 
    * cartboost can do target encode, but this lets us do it for interactions, e.g. target city freq given the 2 countries and affiliate.
    * beware overfitting! 

In [4]:
MIN_TARGET_FREQ = 38 # drop target/city_id values that appear less than this many times, as final step's target 
KEEP_TOP_K_TARGETS = 0 # keep K most frequent city ID targets (redundnat with the above, )

## (some) categorical variables that appear less than this many times will be replaced with a placeholder value!
## Includes CITY id (but done after target filtering, to avoid creating a "rare class" target:
LOW_COUNT_THRESH = 10

RUN_TABNET = False
max_epochs = 5

GET_COUNT_AGG_FEATS = False ## disable getting count, rank etc' groupby features , for speedup

DROP_FIRST_ROW = True

## for matrix factorization/CF:
### morte possible ID_cols :  # last (last step in trip) - would double data per user incorrectly
### hotel_country_lag1 , city_id_lag1  (very relevant - needs shared embeddingm and would increase cardinality a lot.. ) 
ID_COLS = [
#     'device_class',
#            'affiliate_id',
           'booker_country',
           'checkin_quarter',
#            "last",
          "first_hotel_country"] 
MF_KEEP_COLS = ["ID"]+ID_COLS+['city_id',"hotel_country"]

SAVE_TO_DISK = False

TARGET_COL =  'city_id' #"city_id"#'hotel_country' 
USER_ID_COL = "utrip_id"

In [5]:
# most basic categorical columns , without 'user_id', , 'utrip_id' ordevice_class - used for count encoding/filtering
BASE_CAT_COLS = ['city_id',  'affiliate_id', 'booker_country', 'hotel_country']

# ### features to get lags for. Not very robust. May want different feats for lags before -1
# LAG_FEAT_COLS = ['city_id', 'device_class',
#        'affiliate_id', 'booker_country', 'hotel_country', 
#        'duration', 'same_country', 'checkin_day', 'checkin_weekday',
#        'checkin_week',
#         'checkout_weekday','checkout_week',
#        'city_id_count', 'affiliate_id_count',
#        'booker_country_count', 'hotel_country_count', 
#        'checkin_month_count', 'checkin_week_count', 'city_id_nunique',
#        'affiliate_id_nunique', 'booker_country_nunique',
#        'hotel_country_nunique', 'city_id_rank_by_hotel_country',
#        'city_id_rank_by_booker_country', 'city_id_rank_by_affiliate',
#        'affiliate_id_rank_by_hotel_country',
#        'affiliate_id_rank_by_booker_country', 'affiliate_id_rank_by_affiliate',
#        'booker_country_rank_by_hotel_country',
#        'booker_country_rank_by_booker_country',
#        'booker_country_rank_by_affiliate',
#        'hotel_country_rank_by_hotel_country',
#        'hotel_country_rank_by_booker_country',
#        'hotel_country_rank_by_affiliate',
#        'checkin_month_rank_by_hotel_country',
#        'checkin_month_rank_by_booker_country',
#        'checkin_month_rank_by_affiliate']

In [6]:
# https://stackoverflow.com/questions/33907537/groupby-and-lag-all-columns-of-a-dataframe
# https://stackoverflow.com/questions/62924987/lag-multiple-variables-grouped-by-columns
## lag features with groupby over many columns: 
def groupbyLagFeatures(df:pd.DataFrame,lag:[]=[1,2],group="utrip_id",lag_feature_cols=[]):
    """
    lag features with groupby over many columns.
    Assumes sorted data!
    https://stackoverflow.com/questions/62924987/lag-multiple-variables-grouped-by-columns"""
    if len(lag_feature_cols)>0:
        df=pd.concat([df]+[df.groupby(group)[lag_feature_cols].shift(x).add_prefix('lag'+str(x)+"_") for x in lag],axis=1)
    else:
         df=pd.concat([df]+[df.groupby(group).shift(x).add_prefix('lag'+str(x)+"_") for x in lag],axis=1)
    return df


def groupbyFirstLagFeatures(df:pd.DataFrame,group="user_id",lag_feature_cols=[]):
    """
    Get  first/head value lag-like of features with groupby over columns. Assumes sorted data!
    """
    if len(lag_feature_cols)>0:
        df=pd.concat([df]+[df.groupby(group)[lag_feature_cols].transform("first").add_prefix("first_")],axis=1)
    else:
#          df=pd.concat([df]+[df.groupby(group).first().add_prefix("first_")],axis=1)
        df=pd.concat([df]+[df.groupby(group).transform("first").add_prefix("first_")],axis=1)
    return df

######## Get n most popular items, per group
def most_popular(group, n_max=4):
    """Find most popular hotel clusters by destination
    Define a function to get most popular hotels for a destination group.

    Previous version used nlargest() Series method to get indices of largest elements. But the method is rather slow.
    Source: https://www.kaggle.com/dvasyukova/predict-hotel-type-with-pandas
    """
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets


## https://codereview.stackexchange.com/questions/149306/select-the-n-most-frequent-items-from-a-pandas-groupby-dataframe
# https://stackoverflow.com/questions/52073054/group-by-a-column-to-find-the-most-frequent-value-in-another-column
## can get modes (sorted)
# https://stackoverflow.com/questions/50592762/finding-most-common-values-with-pandas-groupby-and-value-counts
## df.groupby('tag')['category'].agg(lambda x: x.value_counts().index[0])
# https://stackoverflow.com/questions/15222754/groupby-pandas-dataframe-and-select-most-common-value
# source2.groupby(['Country','City'])['Short name'].agg(pd.Series.mode)



# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=64,target_col="target"):
    """
    Wrap dataframes with tf.data. 
    This will enable us to use feature columns as a bridge to map from the columns in a dataframe to features used to train the model.
    https://www.tensorflow.org/tutorials/structured_data/feature_columns#create_an_input_pipeline_using_tfdata
    """
    dataframe = dataframe.copy()
    labels = dataframe.pop(target_col)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [7]:
df = pd.read_csv("booking_train_set.csv",
#                  nrows=523456,
                 index_col=[0],
                 parse_dates=["checkin","checkout"],infer_datetime_format=True)

df.sort_values([ "checkin",
                "user_id"],inplace=True)
print(df.nunique())
df

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


user_id           200153
checkin              425
checkout             425
city_id            39901
device_class           3
affiliate_id        3254
booker_country         5
hotel_country        195
utrip_id          217686
dtype: int64


,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
519017,727105,2015-12-31,2016-01-01,18820,mobile,359,The Devilfire Empire,Cobra Island,727105_1
986596,2000964,2015-12-31,2016-01-01,63341,mobile,8151,The Devilfire Empire,Cobra Island,2000964_1
7504,2595109,2015-12-31,2016-01-01,27404,mobile,359,The Devilfire Empire,Cobra Island,2595109_1
52884,110418,2016-01-01,2016-01-02,3763,desktop,9924,The Devilfire Empire,Glubbdubdrib,110418_1
1068508,221863,2016-01-01,2016-01-02,45597,mobile,7774,Gondal,Gondal,221863_1
...,...,...,...,...,...,...,...,...,...
908844,5755992,2017-02-27,2017-02-28,52860,desktop,4568,Gondal,Axphain,5755992_4
1154024,5842454,2017-02-27,2017-02-28,35850,desktop,384,Gondal,Rook Islands,5842454_6
1160065,5936647,2017-02-27,2017-02-28,20199,desktop,4541,Gondal,Kasnia,5936647_2
244942,5955565,2017-02-27,2017-02-28,54384,desktop,8132,Elbonia,Patusan,5955565_2


In [8]:
### i disabled most of thefeature eztraction here for simplicity

# df["duration"] = (df["checkout"] - df["checkin"]).dt.days
# df["same_country"] = (df["booker_country"]==df["hotel_country"]).astype(int)

# df["checkin_day"] = df["checkin"].dt.day
# df["checkin_weekday"] = df["checkin"].dt.weekday
df["checkin_week"] = df["checkin"].dt.isocalendar().week.astype(int) ## week of year
df["checkin_month"] = df["checkin"].dt.month
# df["checkin_year"] = df["checkin"].dt.year-2016

df["checkin_quarter"] = df["checkin"].dt.quarter # relatively redundant but may be used for "id"

# df["checkin_quarter"] = df["checkin_quarter"]/4 # scale. could also do cos, sin extraction. makesi t a float instead of int/embedding


# df["checkout_weekday"] = df["checkout"].dt.weekday
# df["checkout_week"] = df["checkout"].dt.isocalendar().week.astype(int) ## week of year
# df["checkout_day"] = df["checkout"].dt.day ## day of month

## cyclical datetime embeddings
## drop originakl variables? 
## TODO:L add for other variables, +- those that we'll embed (week?)

# df['checkin_weekday_sin'] = np.sin(df["checkin_weekday"]*(2.*np.pi/7))
# df['checkin_weekday_cos'] = np.cos(df["checkin_weekday"]*(2.*np.pi/7))
# df['checkin_month_sin'] = np.sin((df["checkin_month"]-1)*(2.*np.pi/12))
# df['checkin_month_cos'] = np.cos((df["checkin_month"]-1)*(2.*np.pi/12))

# #############
# # last number in utrip id - probably which trip number it is:
# df["utrip_number"] = df["utrip_id"].str.split("_",expand=True)[1].astype(int)

### encode string columns - must be consistent with test data 
### IF we can concat test with train, we can just do a single transformation  for the NON TARGET cols
# obj_cols_list = df.select_dtypes("O").columns.values
obj_cols_list = ['device_class','booker_country','hotel_country',
#                 "city_id"
                ] # we could also define when loading data, dtype

for c in obj_cols_list:
    df[c] = df[c].astype("category")
#     df[c] = df[c].cat.codes.astype(int)
#     print("min",df[c].min()) min is 0 - which is what the embedding (indices) will expect

## view steps of a trip per user & trip, in order. ## last step == 1.
## count #/pct step in a trip (utrip_id) per user. Useful to get the "final" step per trip - for prediction
## note that the order is ascending, so we would need to select by "last" . (i.e "1" is the first step, 2 the second, etc') , or we could use pct .rank(ascending=True,pct=True)
#### this feature overlaps with the count of each trip id (for the final row)
##  = df.sort_values(["checkin","checkout"])... - df already sorted above
df["utrip_steps_from_end"] = df.groupby("utrip_id")["checkin"].rank(ascending=True,pct=True) 

In [9]:
### add features to be consistent with test set of row in trip, and total trips in trip

df["row_num"] = df.groupby("utrip_id")["checkin"].rank(ascending=True,pct=False).astype(int)
utrip_counts = df["utrip_id"].value_counts()
df["total_rows"] = df["utrip_id"].map(utrip_counts)

### last step in trip
df["last"] = (df["total_rows"]==df["row_num"]).astype(int)

df["total_rows"].describe();

### replace rare categorical variable(s) - affiliates
* replace rare variables (under 2 occurrences) with "-1" dummy

In [10]:
### replace rare variables (under 2 occurrences) with "-1" dummy

affiliates_counts = df["affiliate_id"].value_counts()
print("before:", affiliates_counts)
print("uniques",df["affiliate_id"].nunique())
affiliates_counts = affiliates_counts.to_dict()
# df["affiliate_id"] = df["affiliate_id"].where(df["affiliate_id"].apply(lambda x: x.map(x.value_counts()))>=3, -1)
df["affiliate_id"] = df["affiliate_id"].where(df["affiliate_id"].map(affiliates_counts)>=3, -2)
df["affiliate_id"] = df["affiliate_id"].astype(int)

print("after\n",df["affiliate_id"].value_counts())
print("uniques",df["affiliate_id"].nunique())

before: 9924     277775
359      171385
384       88137
9452      85476
4541      41504
          ...  
8351          1
8464          1
2202          1
10513         1
2047          1
Name: affiliate_id, Length: 3254, dtype: int64
uniques 3254
after
 9924    277775
359     171385
384      88137
9452     85476
4541     41504
         ...  
2615         3
5963         3
2618         3
838          3
176          3
Name: affiliate_id, Length: 2152, dtype: int64
uniques 2152


In [11]:
# df["total_rows"].map(pd.cut(df["total_rows"],bins=3,))

#### Add LAG feature(s)  + DROP First visited location rows
* "first" hotel country (vs most recent country visited)
    * `groupbyFirstLagFeatures(df,group="user_id",lag_feature_cols=["hotel_country","city_id"])`
* Can consider: lag1 hotel_country, hotel_id 

In [12]:
## add the "first" place visited/values
### nopte - will need to drop first row in trip, or impute nans when using this feature 

### first by user results in too much sparsity/rareness for our IDs purposes
# df = groupbyFirstLagFeatures(df,group="user_id",lag_feature_cols=["hotel_country"]) # ["hotel_country","city_id"]

## alt - messy, but maybe good enough : 
df = groupbyFirstLagFeatures(df,group=['device_class', 'affiliate_id',
                                       'booker_country','checkin_month',"last"],
                             lag_feature_cols=["hotel_country","city_id",'device_class', 'affiliate_id',
                                              "checkin_quarter","checkin_month","booker_country"])


print(df[["first_hotel_country","hotel_country","city_id","first_city_id"]].nunique())
df

first_hotel_country      144
hotel_country            195
city_id                39901
first_city_id           7863
dtype: int64


,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,checkin_week,checkin_month,checkin_quarter,utrip_steps_from_end,row_num,total_rows,last,first_hotel_country,first_city_id,first_device_class,first_affiliate_id,first_checkin_quarter,first_checkin_month,first_booker_country
519017,727105,2015-12-31,2016-01-01,18820,mobile,359,The Devilfire Empire,Cobra Island,727105_1,53,12,4,0.250,1,4,0,Cobra Island,18820,mobile,359,4,12,The Devilfire Empire
986596,2000964,2015-12-31,2016-01-01,63341,mobile,8151,The Devilfire Empire,Cobra Island,2000964_1,53,12,4,0.200,1,5,0,Cobra Island,63341,mobile,8151,4,12,The Devilfire Empire
7504,2595109,2015-12-31,2016-01-01,27404,mobile,359,The Devilfire Empire,Cobra Island,2595109_1,53,12,4,0.250,1,4,0,Cobra Island,18820,mobile,359,4,12,The Devilfire Empire
52884,110418,2016-01-01,2016-01-02,3763,desktop,9924,The Devilfire Empire,Glubbdubdrib,110418_1,53,1,1,0.100,1,10,0,Glubbdubdrib,3763,desktop,9924,1,1,The Devilfire Empire
1068508,221863,2016-01-01,2016-01-02,45597,mobile,7774,Gondal,Gondal,221863_1,53,1,1,0.125,1,8,0,Gondal,45597,mobile,7774,1,1,Gondal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908844,5755992,2017-02-27,2017-02-28,52860,desktop,4568,Gondal,Axphain,5755992_4,9,2,1,1.000,5,5,1,Gondal,29394,desktop,4568,1,2,Gondal
1154024,5842454,2017-02-27,2017-02-28,35850,desktop,384,Gondal,Rook Islands,5842454_6,9,2,1,1.000,9,9,1,Sylvania,14342,desktop,384,1,2,Gondal
1160065,5936647,2017-02-27,2017-02-28,20199,desktop,4541,Gondal,Kasnia,5936647_2,9,2,1,1.000,12,12,1,Rolisica,8462,desktop,4541,1,2,Gondal
244942,5955565,2017-02-27,2017-02-28,54384,desktop,8132,Elbonia,Patusan,5955565_2,9,2,1,1.000,5,5,1,Elbonia,52327,desktop,8132,1,2,Elbonia


In [13]:
df.nunique()

user_id                  200153
checkin                     425
checkout                    425
city_id                   39901
device_class                  3
affiliate_id               2152
booker_country                5
hotel_country               195
utrip_id                 217686
checkin_week                 53
checkin_month                12
checkin_quarter               4
utrip_steps_from_end        520
row_num                      48
total_rows                   41
last                          2
first_hotel_country         144
first_city_id              7863
first_device_class            3
first_affiliate_id         2152
first_checkin_quarter         4
first_checkin_month          12
first_booker_country          5
dtype: int64

#### Create pseudo ID for use with CF - composed of semi-distinct tuple of variables
* should have moderate uniqueness. Main purpose is to get good embeddings for those variables 

* `'device_class','affiliate_id', 'booker_country','checkin_quarter'` - 14K "uniques"
* `'device_class','affiliate_id', 'booker_country'` - 7.5 K "uniques"

In [14]:
### for possible "user id" embedding/ID : How many unique values are there for these source tuple? :
### Could also maybe add previous location/lag1 country/city ? 
## 'device_class','affiliate_id', 'booker_country' - 7.5 K "uniques"
## 'device_class','affiliate_id', 'booker_country','checkin_month' - 24 K "uniques"
## 'device_class','affiliate_id', 'booker_country','checkin_quarter' 14K "uniques"
## ,"total_rows" 

print(df[ID_COLS + ['checkin_month',"total_rows"]].nunique(axis=0))
df.groupby(ID_COLS).size()

# id2 = [item for item in ID_COLS if item != "last"]
# df.groupby(id2).size()

booker_country           5
checkin_quarter          4
first_hotel_country    144
checkin_month           12
total_rows              41
dtype: int64


booker_country        checkin_quarter  first_hotel_country           
Bartovia              1                Absurdistan                         0
                                       Aldorria                            0
                                       Aldovia                            16
                                       Almaigne                            0
                                       Altis and Stratis, Republic of      3
                                                                        ... 
The Devilfire Empire  4                Yellow Empire                       0
                                       Yerba                               2
                                       Yudonia                           309
                                       Zekistan                            1
                                       Zephyria                            0
Length: 3900, dtype: int64

In [15]:
ID_COLS

['booker_country', 'checkin_quarter', 'first_hotel_country']

In [16]:
df["ID"] = df[ID_COLS].astype(str).sum(1)#.astype("category")

print(df["ID"].value_counts().describe())
print(df["ID"].tail(3))
print(df["ID"].nunique())

count     1411.000000
mean       826.956060
std       2937.318129
min          1.000000
25%          7.000000
50%         42.000000
75%        349.500000
max      55205.000000
Name: ID, dtype: float64
1160065                 Gondal1Rolisica
244942                  Elbonia1Elbonia
787249     The Devilfire Empire1Sokovia
Name: ID, dtype: object
1411


### keep seperate DF of IDs and component features - will make merging id feats simpler later

In [17]:
df_ID = df[["ID"]+ID_COLS].drop_duplicates().set_index("ID")
df_ID

,booker_country,checkin_quarter,first_hotel_country
ID,,,
The Devilfire Empire4Cobra Island,The Devilfire Empire,4,Cobra Island
The Devilfire Empire1Glubbdubdrib,The Devilfire Empire,1,Glubbdubdrib
Gondal1Gondal,Gondal,1,Gondal
Elbonia1Patusan,Elbonia,1,Patusan
The Devilfire Empire1Nova Africa,The Devilfire Empire,1,Nova Africa
...,...,...,...
The Devilfire Empire1Laurania,The Devilfire Empire,1,Laurania
Gondal1San Theodoros,Gondal,1,San Theodoros
Bartovia1Buranda,Bartovia,1,Buranda


* Continue

In [18]:
##### Following aggregation features - would be best to use time window (sort data) to generate, otherwise they will LEAK! (e.g. nunique countries visited)

### count features (can also later add rank inside groups).
### Some may be leaks (# visits in a trip should use time window?) , and do users repeat? 
### can add more counts of group X time period (e.g. affiliate X month of year)

## alt way to get counts/freq :

if GET_COUNT_AGG_FEATS:
    count_cols = [ 'city_id','affiliate_id', 'booker_country', 'hotel_country', 
    #               'utrip_id','user_id', 
     "checkin_month","checkin_week"]
    for c in count_cols:
        df[f"{c}_count"] = df.groupby([c])["duration"].transform("size")

    ########################################################
    ## nunique per trip
    ### https://stackoverflow.com/questions/46470743/how-to-efficiently-compute-a-rolling-unique-count-in-a-pandas-time-series

    nunique_cols = [ 'city_id','affiliate_id', 'booker_country', 'hotel_country']
    # df["nunique_booker_countries"] = df.groupby("utrip_id")["booker_country"].nunique()
    # df["nunique_hotel_country"] = df.groupby("utrip_id")["hotel_country"].nunique()
    for c in nunique_cols:
        df[f"{c}_nunique"] = df.groupby(["utrip_id"])[c].transform("nunique")
    print(df.nunique())

    ########################################################
    ## get frequency/count feature's rank within a group - e.g. within a country (or affiliate) 
    ## add "_count" to column name to get count col name, then add rank col 

    ### ALT/ duplicate feat - add percent rank (instead or in addition)

    rank_cols = ['city_id','affiliate_id', 'booker_country','hotel_country',
     "checkin_month"]
    ### what is meaning of groupby and rank of smae variable by same var? Surely should be 1 / unary? 
    for c in rank_cols:
        df[f"{c}_rank_by_hotel_country"] = df.groupby(['hotel_country'])[f"{c}_count"].transform("rank")
        df[f"{c}_rank_by_booker_country"] = df.groupby(['booker_country'])[f"{c}_count"].transform("rank")
        df[f"{c}_rank_by_affiliate"] = df.groupby(['affiliate_id'])[f"{c}_count"].transform("rank")     
else:
    freq = df["city_id"].value_counts()
    df["city_id_count"] = df["city_id"].map(freq)
    print(freq.describe())
    
df

count    39901.000000
mean        29.243252
std        218.801654
min          1.000000
25%          1.000000
50%          3.000000
75%          9.000000
max      11242.000000
Name: city_id, dtype: float64


,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,checkin_week,checkin_month,checkin_quarter,utrip_steps_from_end,row_num,total_rows,last,first_hotel_country,first_city_id,first_device_class,first_affiliate_id,first_checkin_quarter,first_checkin_month,first_booker_country,ID,city_id_count
519017,727105,2015-12-31,2016-01-01,18820,mobile,359,The Devilfire Empire,Cobra Island,727105_1,53,12,4,0.250,1,4,0,Cobra Island,18820,mobile,359,4,12,The Devilfire Empire,The Devilfire Empire4Cobra Island,535
986596,2000964,2015-12-31,2016-01-01,63341,mobile,8151,The Devilfire Empire,Cobra Island,2000964_1,53,12,4,0.200,1,5,0,Cobra Island,63341,mobile,8151,4,12,The Devilfire Empire,The Devilfire Empire4Cobra Island,196
7504,2595109,2015-12-31,2016-01-01,27404,mobile,359,The Devilfire Empire,Cobra Island,2595109_1,53,12,4,0.250,1,4,0,Cobra Island,18820,mobile,359,4,12,The Devilfire Empire,The Devilfire Empire4Cobra Island,3614
52884,110418,2016-01-01,2016-01-02,3763,desktop,9924,The Devilfire Empire,Glubbdubdrib,110418_1,53,1,1,0.100,1,10,0,Glubbdubdrib,3763,desktop,9924,1,1,The Devilfire Empire,The Devilfire Empire1Glubbdubdrib,5544
1068508,221863,2016-01-01,2016-01-02,45597,mobile,7774,Gondal,Gondal,221863_1,53,1,1,0.125,1,8,0,Gondal,45597,mobile,7774,1,1,Gondal,Gondal1Gondal,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908844,5755992,2017-02-27,2017-02-28,52860,desktop,4568,Gondal,Axphain,5755992_4,9,2,1,1.000,5,5,1,Gondal,29394,desktop,4568,1,2,Gondal,Gondal1Gondal,12
1154024,5842454,2017-02-27,2017-02-28,35850,desktop,384,Gondal,Rook Islands,5842454_6,9,2,1,1.000,9,9,1,Sylvania,14342,desktop,384,1,2,Gondal,Gondal1Sylvania,692
1160065,5936647,2017-02-27,2017-02-28,20199,desktop,4541,Gondal,Kasnia,5936647_2,9,2,1,1.000,12,12,1,Rolisica,8462,desktop,4541,1,2,Gondal,Gondal1Rolisica,12
244942,5955565,2017-02-27,2017-02-28,54384,desktop,8132,Elbonia,Patusan,5955565_2,9,2,1,1.000,5,5,1,Elbonia,52327,desktop,8132,1,2,Elbonia,Elbonia1Elbonia,526


In [19]:
print("cities with more than 7 occurences:",df.loc[df["city_id_count"]>=7]["city_id"].nunique())
print(f"cities with more than MIN_TARGET_FREQ ({MIN_TARGET_FREQ}) occurences:",df.loc[df["city_id_count"]>=MIN_TARGET_FREQ]["city_id"].nunique())
print(f"rows left if filtering by MIN_TARGET_FREQ :",df.loc[df["city_id_count"]>=MIN_TARGET_FREQ].shape[0])

cities with more than 7 occurences: 12345
cities with more than MIN_TARGET_FREQ (38) occurences: 3498
rows left if filtering by MIN_TARGET_FREQ : 970972


In [20]:
assert df.isna().sum().max() ==0

In [21]:
# df2 = df[["user_id","city_id"]].drop_duplicates().copy()
df2 = df.drop_duplicates(subset=["user_id","city_id"])["city_id"].copy()
print(df2.shape[0])
print("df2 nunique (cities without duplicate user visits)",df2.nunique())

# c2_counts = df2["city_id"].value_counts()
c2_counts = df2.value_counts()
# df2["new_counts"] = df2["city_id"].map(c2_counts)
# df2["new_counts"] = df2.map(c2_counts)
print("city counts")
print(c2_counts)
print(c2_counts.describe())
print("cities with at least 2:",(c2_counts>=2).sum())
print("cities with at least 3:",(c2_counts>=3).sum())
print("cities with at least 5:",(c2_counts>=5).sum())
print("cities with at least 10:",(c2_counts>=10).sum())
print("cities with at least 15:",(c2_counts>=15).sum())
print("cities with at least 20:",(c2_counts>=20).sum())
print("cities with at least 30:",(c2_counts>=30).sum())
print("cities with at least 50:",(c2_counts>=50).sum())
print("cities with at least 100:",(c2_counts>=100).sum())

c2_freq = df2.value_counts(normalize=True)
print("top 4 sum coverage (normalized): ",c2_freq[0:4].sum().round(3))
print("top 50 sum coverage (normalized): ",c2_freq[0:50].sum().round(3))
print("top 100 sum coverage (normalized): ",c2_freq[0:100].sum().round(3))
print("top 400 sum coverage (normalized): ",c2_freq[0:400].sum().round(3))
print("top 1,000 sum coverage (normalized): ",c2_freq[0:1000].sum().round(3))
print("top 5,000 sum coverage (normalized): ",c2_freq[0:5000].sum().round(3))
print("top 8,000 sum coverage (normalized): ",c2_freq[0:8000].sum().round(3))
print("top 15,000 sum coverage (normalized): ",c2_freq[0:15000].sum().round(3))

1029804
df2 nunique (cities without duplicate user visits) 39901
city counts
23921    8137
55128    7197
47499    7188
64876    6724
29319    6361
         ... 
50916       1
57063       1
46826       1
38638       1
2049        1
Name: city_id, Length: 39901, dtype: int64
count    39901.000000
mean        25.808977
std        173.750203
min          1.000000
25%          1.000000
50%          3.000000
75%          9.000000
max       8137.000000
Name: city_id, dtype: float64
cities with at least 2: 27533
cities with at least 3: 21337
cities with at least 5: 15248
cities with at least 10: 9281
cities with at least 15: 6836
cities with at least 20: 5499
cities with at least 30: 4006
cities with at least 50: 2673
cities with at least 100: 1550
top 4 sum coverage (normalized):  0.028
top 50 sum coverage (normalized):  0.183
top 100 sum coverage (normalized):  0.263
top 400 sum coverage (normalized):  0.481
top 1,000 sum coverage (normalized):  0.64
top 5,000 sum coverage (normalized):  0.8

## Frequent city target List + City count encoding
* Get the K most frequent target city IDs - selected based on frequency as final destination (not just overall)
* +- Also after this, replace rare city IDs categorical features with count encoding to reduce dimensionality
    * Keep them as count, or aggregate all of them as "under_K"?

##### Output  : `TOP_TARGETS` - filter data by this *after* creation of lag features ! 

* Drop duplicates by the same user (reduce possible bias of frequent users? Only relevant if test is seperater from "frequent travellers") 
    * results in 216,633 , vs 217,686 without dropping duplicates by users
    * ~19.9k unique cities
    
* Could do other encodings - https://contrib.scikit-learn.org/category_encoders/count.html

* Note that all this is after we've added rank, count features beforehand, so that information won't be lost for these variables, despite these transforms



* **NOTE** he most frequent final destinations are NOT the same as the most popular overall destinations +- first location ! 

In [22]:
if KEEP_TOP_K_TARGETS > 0 :
    df_end = df.loc[df["utrip_steps_from_end"]==1].drop_duplicates(subset=["city_id","hotel_country","user_id"])[["city_id","hotel_country"]].copy()
    print(df_end.shape[0])
    end_city_counts = df_end.city_id.value_counts()
    print(end_city_counts)
    
    TOP_TARGETS = end_city_counts.head(KEEP_TOP_K_TARGETS).index.values
    print(f"top {KEEP_TOP_K_TARGETS} targets \n",TOP_TARGETS)
    
#     assert df.loc[df["city_id"].isin(TOP_TARGETS)]["city_id"].nunique() == KEEP_TOP_K_TARGETS

####        
# replace low frequency categoircal features    

# ##replace with count encoding if have at least k, group rarest as "-1":# df[BASE_CAT_COLS] = df[BASE_CAT_COLS].where(df[BASE_CAT_COLS].apply(lambda x: x.map(x.value_counts()))>=LOW_COUNT_THRESH, -1)   
# ## replace/group only the rare variables : 
# df[BASE_CAT_COLS] = df[BASE_CAT_COLS].where(df[BASE_CAT_COLS].apply(lambda x: x.map(x.value_counts()))>=LOW_COUNT_THRESH, -1)
# df[BASE_CAT_COLS].head()

##### Long tail of targets warning!
* 75% of cities appear less than 4 times in the data (as a final destination!) 
    * Dropping them will mean a maximum accuracy of 25% at best!!
    * training on intermediates may help overcome improve this. 
* Using ~2d step+ , still leaves us with 75% appearing less than 7 times

* Top 4,000 cities (just for those as final trip destination) - offers 89% coverage - 

* Unsure how to handle this - too amny targets to learn, and no auxiliary data to help learn it? 

In [23]:
# # df_end.city_id.value_counts(normalize=True)[0:4000].sum().round(3)# 4k: 89% coverage  (note, this is just for the end count cities, not all cities overall)

# df_end.city_id.value_counts(normalize=True)[0:2000].sum().round(3) #7k: 97% coverage

### Filter data by city id frequency
#### df2 - smaller df (may not ben ecessary to make

* drop rows if it's city id appears less than X times - this is prior to CF
* We could also add inclusion/exclusion based on target appearing/frequency as target in final stage of rtip - optional
* maybe also drop (end exclude in freq counting) thefirst point in a trip ? 



In [24]:
### unsure about this filtering - depends if data points are real or mistake
print("dropping users with less than 4 trips")
df2 = df.loc[df["total_rows"]>=4]#.copy()
# print("abnormal users dropped",df.shape[0]-df2.shape[0])

print(f"dropping cities with less than {MIN_TARGET_FREQ} occurences:")
df2 = df2.loc[df2.groupby(["city_id"])["hotel_country"].transform("count")>=MIN_TARGET_FREQ] ## update count
# df2 = df2.loc[df2["city_id_count"]>=MIN_TARGET_FREQ]
print(df2.shape[0])

# print(f"dropping users with less than 4 instances, after previous city filter:")
df2 = df2.loc[df2.groupby(["utrip_id"])[TARGET_COL].transform("count")>=4]

# print(f"dropping cities with less than {MIN_TARGET_FREQ} occurences:")
df2 = df2.loc[df2.groupby(["city_id"])["hotel_country"].transform("count")>=MIN_TARGET_FREQ]

print(f"dropping users with less than 4 instances, after previous filters:")
df2 = df2.loc[df2.groupby(["utrip_id"])[TARGET_COL].transform("count")>=4]

df2 = df2.loc[df2.groupby(["city_id"])["hotel_country"].transform("count")>=MIN_TARGET_FREQ]
df2 = df2.loc[df2.groupby(["utrip_id"])[TARGET_COL].transform("count")>=4]
df2 = df2.loc[df2.groupby(["city_id"])["hotel_country"].transform("count")>=MIN_TARGET_FREQ]
df2 = df2.loc[df2.groupby(["utrip_id"])[TARGET_COL].transform("count")>=4]
df2 = df2.loc[df2.groupby(["city_id"])["hotel_country"].transform("count")>=MIN_TARGET_FREQ]
df2 = df2.loc[df2.groupby(["utrip_id"])[TARGET_COL].transform("count")>=4]

print("rows left:",df2.shape[0])

print("nunique cities after freq filt",df2[["city_id","utrip_id","user_id","hotel_country"]].nunique())
print("nunique city_id per hotel_country:")
df2.groupby(["hotel_country"])["city_id"].nunique().describe()

dropping users with less than 4 trips
dropping cities with less than 38 occurences:
968939
dropping users with less than 4 instances, after previous filters:
rows left: 788514
nunique cities after freq filt city_id            2530
utrip_id         150653
user_id          141625
hotel_country        96
dtype: int64
nunique city_id per hotel_country:


count    195.000000
mean      12.974359
std       36.190139
min        0.000000
25%        0.000000
50%        0.000000
75%        7.000000
max      303.000000
Name: city_id, dtype: float64

In [25]:
df = df2

* drop the first location visited per trip, as that shares the "first country/hotel/city id feature" ??

In [26]:
if DROP_FIRST_ROW: ## drop the first location visited per trip, (row_num = 1) from df data/interactions
    df = df.loc[df["row_num"]>1]
    print(df.shape[0])

645813


### LightFM basic user-item CF
* just users and country without side features
 * todo: train/test split by last? 
 

In [27]:
from datetime import datetime, timedelta
from sklearn import preprocessing
from lightfm import LightFM
from scipy.sparse import csr_matrix 
from scipy.sparse import coo_matrix 
from sklearn.metrics import roc_auc_score
import time
from lightfm.evaluation import auc_score
import seaborn as sns

from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
from lightfm.cross_validation import random_train_test_split 

/opt/anaconda3/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [28]:
# %%time
# df_interactions = pd.crosstab(df[USER_ID_COL],df[TARGET_COL]).stack().reset_index()
# ## stack so that target wilk l be count/sum of interactions /weight
# df_interactions.rename(columns={0:"target"},inplace=True)

# print(df_interactions.describe())
# print(df_interactions[[USER_ID_COL,TARGET_COL]].nunique())
# df_interactions

In [29]:
### lightFM dataset
## https://stackoverflow.com/questions/51144061/how-to-create-a-sparsematrix-from-dataframe-in-a-specific-format
## https://github.com/jamesdhope/recommender/blob/master/recommender.py
## https://www.kaggle.com/niyamatalmass/lightfm-hybrid-recommendation-system#Defining-our-necessary-functions  - funcs for prepping features
dataset = Dataset(user_identity_features=True, item_identity_features=True)

# dataset.fit(df[USER_ID_COL].unique(), df[TARGET_COL].unique())
dataset.fit(df[USER_ID_COL], df[TARGET_COL])

num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 150653, num_items 2530.


In [30]:
# assert num_users == df_interactions[USER_ID_COL].nunique()
# assert num_items == df_interactions[TARGET_COL].nunique()

In [31]:
interactions_matrix, weights_matrix = dataset.build_interactions([tuple(i) for i in df[[USER_ID_COL,TARGET_COL]].values])
# interactions_matrix, weights_matrix = dataset.build_interactions([tuple(i) for i in df_interactions.values])

In [32]:
(train, test) = random_train_test_split(interactions=interactions_matrix, test_percentage=0.2,random_state=7)

In [33]:
print(repr(interactions_matrix))

<150653x2530 sparse matrix of type '<class 'numpy.int32'>'
	with 645813 stored elements in COOrdinate format>


In [ ]:
interactions_matrix

In [34]:
## https://www.dropbox.com/sh/o4tio3ik9bvx0ve/AACt1zuOSM0KaYXaL_AwugT-a?dl=0&fbclid=IwAR1OIebCTYv2ebi_VI5MIBVgSsQ4JwtyTYAt4jRPDlVHamB9iCBpJF7OBWw&preview=ugly_code.py
def create_feature_list(df_,cols):
    feature_list = []
    for col in cols:
        feature_list += list(set(df_[col].to_list()))
    return feature_list
  

def build_features(df_):
    return [(row[0],
             [row[i] for i in range(1,len(row))]) 
            for row in df_.itertuples(index=False)]


def sample_lightfm_recommendation(model, train,items_labels,user_ids,item_features=None,user_features=None,topk=5):
    """
    Get topk recommendations per user
    """

    #number of users and movies in training data
    n_users, n_items = train.shape

    #generate recommendations for each user we input
    for user_id in user_ids:

        #movies they already like
        #known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        known_positives = items_labels[train.tocsr()[user_id].indices]
        
        #movies our model predicts they will like
        scores = model.predict(user_id, np.arange(n_items),user_features=user_features,item_features=item_features)
        #rank them in order of most liked to least
        top_items = items_labels[np.argsort(-scores)]

        #print out the results
        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)


In [35]:
df.columns

Index(['user_id', 'checkin', 'checkout', 'city_id', 'device_class',
       'affiliate_id', 'booker_country', 'hotel_country', 'utrip_id',
       'checkin_week', 'checkin_month', 'checkin_quarter',
       'utrip_steps_from_end', 'row_num', 'total_rows', 'last',
       'first_hotel_country', 'first_city_id', 'first_device_class',
       'first_affiliate_id', 'first_checkin_quarter', 'first_checkin_month',
       'first_booker_country', 'ID', 'city_id_count'],
      dtype='object')

In [36]:
df["user_id"] = "us_" + df["user_id"].astype(str)

df["first_affiliate_id"] = "a_" + df["first_affiliate_id"].astype(str)
df["first_city_id"] = "c_" + df["first_city_id"].astype(str)
df["first_checkin_month"] = "m_" + df["first_checkin_month"].astype(str)
df["first_checkin_quarter"] = "q_" + df["first_checkin_quarter"].astype(str)

<ipython-input-36-c3dcab40c905>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["user_id"] = "us_" + df["user_id"].astype(str)
<ipython-input-36-c3dcab40c905>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["first_affiliate_id"] = "a_" + df["first_affiliate_id"].astype(str)
<ipython-input-36-c3dcab40c905>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [37]:
user_feats = [ 
    'first_hotel_country', 
              "first_booker_country",
#        'first_city_id', 
#               'first_device_class', 
#               'first_affiliate_id', 
#        'first_checkin_quarter', 
#             'first_checkin_month',
#               "user_id"
             ]

## ALT: 
# user_feats = ["ID"]

users = df[['utrip_id'] +  user_feats].drop_duplicates(["utrip_id"])
print(users.nunique())
users

utrip_id                150653
first_hotel_country        118
first_booker_country         5
dtype: int64


,utrip_id,first_hotel_country,first_booker_country
52885,110418_1,Glubbdubdrib,The Devilfire Empire
1068509,221863_1,Vadeem,Tcherkistan
400623,403238_1,Cobra Island,Elbonia
761855,414381_1,Nova Africa,The Devilfire Empire
1002561,870848_1,Bartovia,The Devilfire Empire
...,...,...,...
268836,6140090_2,Fook Island,Elbonia
572120,181301_1,Gondal,Gondal
877422,3209267_5,Kasnia,Elbonia
378261,4682726_1,Samavia,Gondal


In [38]:
df_item_features = df[["city_id","hotel_country"]].drop_duplicates()
print(df_item_features.nunique())
df_item_features

city_id          2530
hotel_country      96
dtype: int64


,city_id,hotel_country
52885,3763,Glubbdubdrib
1068509,36063,Gondal
400623,40871,Patusan
761855,2813,Nova Africa
1002561,51128,Bartovia
...,...,...
205125,14117,Leutonia
742138,29672,Franchia
760325,7458,Novistrana
755388,43540,Bahavia


In [39]:
# features_list = create_feature_list(items,cols=['section_primary','writer_name'])
# user_features_list = create_feature_list(users,cols=['popular_section','popular_platform','popular_sources'])

features_list = create_feature_list(df_item_features,cols=["hotel_country"])
user_features_list = create_feature_list(users,cols=user_feats)


In [40]:
if TARGET_COL !=  'hotel_country':  ## no item level features for country
    dataset.fit_partial(items=df_item_features.city_id,
                   item_features=features_list,
                     users=users["utrip_id"],
                    user_features=user_features_list, # ORIG
#                         user_features = user_features
                   )    
else: 
    dataset.fit_partial(users=users["utrip_id"],
                        user_features=user_features_list)

In [41]:
user_features = dataset.build_user_features(build_features(users), normalize=False)

if TARGET_COL !=  'hotel_country':
    item_features = dataset.build_item_features(build_features(df_item_features))

In [42]:
type(user_features)

scipy.sparse.csr.csr_matrix

LightFM makes certain subtle assumptions when you do or do not pass side information. 
When no user_features or item_features are explicitly included, then LightFM assumes that both feature matrices are in fact identity matrices of size (num_users X num_users) or (num_items X num_items) for user and item feature matrices, respectively. What this is effectively doing is one-hot-encoding each user and item ID as a single feature vector.
In the case where you do pass an item_features matrix, then LightFM does not do any one-hot-encoding. Thus, each user and item ID does not get its own vector unless you explicitly define one.
The easiest way to do this is to make your own identity matrix and stack it on the side of the item_features matrix that we already created. This way, each item is described by a single vector for its unique ID and then a set of vectors for each of its tags.

*  https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/

* **NOTE** - Apparently not needed anymore, `dataset` function creates latent vectors by default ? 
    * https://making.lyst.com/lightfm/docs/lightfm.data.html

In [43]:
# ### add item level features ohe - https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/
# import scipy as sp
# # Need to hstack user_features


# eye = sp.sparse.eye(user_features.shape[0], user_features.shape[0]).tocsr() ## 212 GB if not sparse
# user_features_concat = sp.sparse.hstack((eye, user_features))
# print(type(user_features_concat))
# user_features_concat = user_features_concat.tocsr().astype(np.float32)
# print(type(user_features_concat))

# ## new : 
# # eye = sp.sparse.eye(user_features.shape[0], user_features.shape[0]).tocsr()
# # user_features = sp.hstack((eye, user_features))
# # user_features = user_features.tocsr().astype(np.float32)

# # user_features_concat = sp.sparse.csr_matrix(user_features_concat)
# # print(type(user_features_concat))


In [44]:
# if TARGET_COL ==  'hotel_country':  ## no item level features for country ? 
#     dataset.fit_partial(users=users["utrip_id"],
#     #                     user_features=user_features_concat, ## error : TypeError: unhashable type: 'csr_matrix'
#                         user_features=user_features_list,
#                        )
# else: 
#         dataset.fit_partial(items=df_item_features.city_id,
#                        item_features=features_list,
#                          users=users["utrip_id"],
#     #                     user_features=user_features_concat, ## error : TypeError: unhashable type: 'csr_matrix'
#                         user_features=user_features_list,
#                        )

In [45]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 150653, num_items 2530.


In [46]:
### - modifies and uses user/item features https://gist.github.com/kayibal/16340660d1d85b9ea1872a5d9be0f383
# ## error: 
# item_features = dataset.build_item_features([tuple(i) for i in df_item_features.values])


In [47]:
%%time
model = LightFM(no_components=16,loss='warp',n=20,max_sampled=30, item_alpha = 1.6e-08, user_alpha = 1.8e-09)

# (train, test) = random_train_test_split(interactions=interactions_matrix, test_percentage=0.2)

model.fit(train,
#           item_features=item_features, user_features=user_features, # ORIG
          item_features=item_features, user_features=user_features, ## are these the right inputs ??? 
          epochs=10,
          verbose=True,num_threads=14   
         )

Epoch: 100%|██████████| 10/10 [00:11<00:00,  1.13s/it]

CPU times: user 11.3 s, sys: 54.2 ms, total: 11.3 s
Wall time: 11.3 s


In [48]:
%%time
train_precision_4 = precision_at_k(model, train,k=4,num_threads=7,item_features=item_features,user_features=user_features).mean()
test_precision_4 = precision_at_k(model, test,k=4,num_threads=7,item_features=item_features,user_features=user_features).mean()

train_recall_4 = recall_at_k(model, train,k=4,num_threads=7,item_features=item_features,user_features=user_features).mean()
test_recall_4 = recall_at_k(model, test,k=4,num_threads=7,item_features=item_features,user_features=user_features).mean()
print('Recall @4: train %.2f, test %.2f.' % (train_recall_4, test_recall_4))

train_recall_90 = recall_at_k(model, train,k=90,num_threads=7,item_features=item_features,user_features=user_features).mean()
test_recall_90 = recall_at_k(model, test,k=90,num_threads=7,item_features=item_features,user_features=user_features).mean()
print('Recall @90: train %.2f, test %.2f.' % (train_recall_90, test_recall_90))

train_auc = auc_score(model, train,num_threads=7,
                     item_features=item_features,user_features=user_features).mean()
test_auc = auc_score(model, test,num_threads=7,
                     item_features=item_features,user_features=user_features
                    ).mean()

# print('Precision @10: train %.2f, test %.2f.' % (train_precision_10, test_precision_10))
print('Precision @4: train %.2f, test %.2f.' % (train_precision_4, test_precision_4))
# print('Recall @30: test %.2f.' % ( recall_at_k(model, test,k=30,num_threads=8,item_features=item_features,user_features=user_features).mean()))

print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

## 20 epochs, 64 components, min 20 target freq:
# Precision @4: train 0.35, test 0.14.
# Recall @4: train 0.35, test 0.37.
# Recall @90: train 0.83, test 0.87.
##  Recall @30:  test 
# AUC: train 0.99, test 0.99.

Recall @4: train 0.38, test 0.37.
Recall @90: train 0.88, test 0.90.
Precision @4: train 0.31, test 0.13.
AUC: train 0.99, test 0.99.
CPU times: user 1min 38s, sys: 98.4 ms, total: 1min 38s
Wall time: 1min 38s


In [49]:
# train_recall_4 = recall_at_k(model, train,train,k=4,num_threads=7).mean()
# test_recall_4 = recall_at_k(model, test,train,k=4,num_threads=7).mean()
# print('Recall @4: train %.2f, test %.2f.' % (train_recall_4, test_recall_4))


In [50]:
# ### just item level features  (for country level - identical to just pure CF)
# model_item = LightFM(no_components=16,loss='warp',n=20,max_sampled=30, item_alpha = 1.6e-08, user_alpha = 1.8e-09)
# model_item.fit(train, epochs=10,item_features=item_features,verbose=True,num_threads=8)

# train_precision_4 = precision_at_k(model_item, train,k=4,num_threads=7,item_features=item_features).mean()
# test_precision_4 = precision_at_k(model_item, test,k=4,num_threads=7,item_features=item_features).mean()

# train_recall_4 = recall_at_k(model_item, train,k=4,num_threads=7,item_features=item_features).mean()
# test_recall_4 = recall_at_k(model_item, test,k=4,num_threads=7,item_features=item_features).mean()

# train_recall_90 = recall_at_k(model_item, train,k=90,num_threads=7,item_features=item_features).mean()
# test_recall_90 = recall_at_k(model_item, test,k=90,num_threads=7,item_features=item_features).mean()
# print('Recall @90: train %.2f, test %.2f.' % (train_recall_90, test_recall_90))

# train_auc = auc_score(model_item, train,num_threads=7,
#                      item_features=item_features).mean()
# test_auc = auc_score(model_item, test,num_threads=7,
#                      item_features=item_features
#                     ).mean()

# # print('Precision @10: train %.2f, test %.2f.' % (train_precision_10, test_precision_10))
# print('Precision @4: train %.2f, test %.2f.' % (train_precision_4, test_precision_4))
# print('Recall @4: train %.2f, test %.2f.' % (train_recall_4, test_recall_4))
# print('Recall @30: test %.2f.' % ( recall_at_k(model_item, test,k=30,num_threads=8,item_features=item_features).mean()))

# print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

# ## just item feats , 20 epochs, 64 components, min 20 target freq:
# # Precision @4: train 0.45, test 0.15.
# # Recall @4: train 0.46, test 0.40.

# # Recall @90: train 0.89, test 0.90.
# # AUC: train 1.00, test 0.99.

# ##  Recall @30: test 0.783 (not sure if with less components ?)

In [51]:
# ## model_base performnce evaluation - without user/item features - does worse than + item level
model_base = LightFM(no_components=16,loss='warp',n=20,max_sampled=30, item_alpha = 1.6e-08, user_alpha = 1.8e-09)
model_base.fit(train, epochs=10,num_threads=14)

train_precision_4 = precision_at_k(model_base, train,k=4,num_threads=8).mean()
test_precision_4 = precision_at_k(model_base, test,k=4,num_threads=8).mean()
print('Precision @4: train %.2f, test %.2f.' % (train_precision_4, test_precision_4))

train_recall_4 = recall_at_k(model_base, train,k=4,num_threads=8).mean()
test_recall_4 = recall_at_k(model_base, test,k=4,num_threads=8).mean()
print('Recall @4: train %.2f, test %.2f.' % (train_recall_4, test_recall_4))

train_recall_90 = recall_at_k(model_base, train,k=90,num_threads=7).mean()
test_recall_90 = recall_at_k(model_base, test,k=90,num_threads=7).mean()
print('Recall @90: train %.2f, test %.2f.' % (train_recall_90, test_recall_90))

train_auc = auc_score(model_base, train,num_threads=12).mean()
test_auc = auc_score(model_base, test,num_threads=12,).mean()
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

# # Precision @4: train 0.24, test 0.09.
# # Recall @4: train 0.23, test 0.24.
# # Recall @90: train 0.62, test 0.65.
# # AUC: train 0.93, test 0.92.

Precision @4: train 0.38, test 0.14.
Recall @4: train 0.48, test 0.42.
Recall @90: train 0.91, test 0.92.
AUC: train 1.00, test 0.99.


In [52]:
%%time
### just user level side-features
model_user = LightFM(no_components=16,loss='warp',n=20,max_sampled=30, item_alpha = 1.6e-08, user_alpha = 1.8e-09)
model_user.fit(train, epochs=10,user_features=user_features,verbose=True,num_threads=8)

train_precision_4 = precision_at_k(model_user, train,k=4,num_threads=14,user_features=user_features).mean()
test_precision_4 = precision_at_k(model_user, test,k=4,num_threads=14,user_features=user_features).mean()
print('Precision @4: train %.2f, test %.2f.' % (train_precision_4, test_precision_4))
train_recall_4 = recall_at_k(model_user, train,k=4,num_threads=14,user_features=user_features).mean()
test_recall_4 = recall_at_k(model_user, test,k=4,num_threads=14,user_features=user_features).mean()
print('Recall @4: train %.2f, test %.2f.' % (train_recall_4, test_recall_4))

train_recall_90 = recall_at_k(model_user, train,k=90,num_threads=14,user_features=user_features).mean()
test_recall_90 = recall_at_k(model_user, test,k=90,num_threads=14,user_features=user_features).mean()
print('Recall @90: train %.2f, test %.2f.' % (train_recall_90, test_recall_90))

train_auc = auc_score(model_user, train,num_threads=14,
                     user_features=user_features).mean()
test_auc = auc_score(model_user, test,num_threads=14,
                     user_features=user_features
                    ).mean()
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))
# print('Precision @10: train %.2f, test %.2f.' % (train_precision_10, test_precision_10))

# print('Recall @30: test %.2f.' % ( recall_at_k(model_user, test,k=30,num_threads=8,user_features=user_features).mean()))


Epoch: 100%|██████████| 10/10 [00:08<00:00,  1.20it/s]


Precision @4: train 0.34, test 0.11.
Recall @4: train 0.42, test 0.33.
Recall @90: train 0.85, test 0.81.
AUC: train 0.99, test 0.97.
CPU times: user 1min 25s, sys: 104 ms, total: 1min 25s
Wall time: 1min 25s


* hyperparam search
    *  https://stackoverflow.com/questions/49896816/how-do-i-optimize-the-hyperparameters-of-lightfm

In [53]:
### https://stackoverflow.com/questions/49896816/how-do-i-optimize-the-hyperparameters-of-lightfm
import itertools
import numpy as np

from lightfm import LightFM
from lightfm.evaluation import auc_score


def sample_hyperparameters():
    """
    Yield possible hyperparameter choices.
    """

    while True:
        yield {
            "no_components": np.random.randint(8, 64),
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),
            "loss": np.random.choice(["bpr", "warp", "warp-kos"]),
            "learning_rate": np.random.exponential(0.05),
            "item_alpha": np.random.exponential(1e-8),
            "user_alpha": np.random.exponential(1e-8),
#             "max_sampled": np.random.randint(5, 40),
#             "num_epochs": np.random.randint(1, 8),
            "num_epochs":6
        }

def random_search(train, test, num_samples=10, num_threads=8):
    """
    Sample random hyperparameters, fit a LightFM model, and evaluate it
    on the test set.

    Parameters
    ----------

    train: np.float32 coo_matrix of shape [n_users, n_items]
        Training data.
    test: np.float32 coo_matrix of shape [n_users, n_items]
        Test data.
    num_samples: int, optional
        Number of hyperparameter choices to evaluate.


    Returns
    -------

    generator of (auc_score, hyperparameter dict, fitted model)

    """

    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")
        model = LightFM(**hyperparams) # ,learning_rate=.03
        model.fit(train, epochs=num_epochs, num_threads=num_threads,
                 item_features=item_features, user_features=user_features)
        ### should i pass in train_interactions (when i have repeats) ? 
#         score = auc_score(model, test, train_interactions=train, num_threads=num_threads).mean() # ORIG
#         score = precision_at_k(model, test,  num_threads=num_threads,k=4).mean()
        score = recall_at_k(model, test,  num_threads=num_threads,k=30
                           ,item_features=item_features,user_features=user_features).mean()

        hyperparams["num_epochs"] = num_epochs

        yield (score, hyperparams, model)


In [54]:
%%time
# (score, hyperparams, model) = max(random_search(train, test, num_threads=8, num_samples=25), key=lambda x: x[0])
# print("Best score {} at {}".format(score, hyperparams))

### Best score - Recall @30: 0.747 at {'no_components': 37, 'learning_schedule': 'adagrad', 'loss': 'warp-kos', 'learning_rate': 0.192, 'item_alpha': 1.669e-08, 'user_alpha': 1.88e-09, 'num_epochs': 6}

### raw model search (no side-feats): (P@4): Best score 0.1326 at {'no_components': 60, 'learning_schedule': 'adagrad', 'loss': 'warp', 'learning_rate': 0.029032455085289472, 'num_epochs': 6}

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


### add lag features + Train/test/data split
* Lag feats (remember for categorical)
* Drop leak features (target values - country, city)

* drop instances  that lack history (e.g. at least 3d step and onwards) - by dropna in lag feat
* fill nans
* Split train/test by `user id` / split could maybe be by `utrip ID` ? ? 
    * Test - only last trip
    *  stratified train/test split by class - then drop any train rows with overlap with tests' IDs.  
        * Could also stratify by users, but risks some classes being non present in test
        
###### Big possible improvement to lag features: Have "first location" (starting point) "lag" feature
* `groupbyFirstLagFeatures(df,group="user_id",lag_feature_cols=["hotel_country","city_id"])`

#### Artifical train/test split grouping 

* Make additional grouping for train/test split.
WARNING: This is NOT as good/reliable as splitting by USER_ID or utripID, but we do this for now, since we don't have that variable here
* This splitting would result in some variables not being learned ? 

* Ignore some of the ID columns for this groups creation
    * warning - manua/fragile/may break if columns used change
    
This splitting is **not reliable**!! It is just to try to approximate something like splitting across trips

* Mainly - groups should include rows with last 0/1 for same trip
    * How bad is it that we group/exclude by city_id (for a specific grouping)  ?? 
    * Alt - could do group also with lag1_city_id ? (If using that)

In [55]:
df_feat.columns

NameError: name 'df_feat' is not defined

In [ ]:
df_feat["Group_id"] = df_feat[[ 'device_class', 'affiliate_id', 'booker_country',
                                f"{TARGET_COL}", #"hotel_country",  # 'city_id',
                               'checkin_quarter']].astype(str).sum(1)
print(df_feat["Group_id"].nunique())

In [ ]:
train_inds, test_inds = next(GroupShuffleSplit(test_size=.25, n_splits=2, random_state = 7).split(df_feat, groups=df_feat['Group_id']))

train = df_feat.iloc[train_inds].drop("Group_id",axis=1)
test = df_feat.iloc[test_inds]

## split test into validation and final test (only last stage of trip). We could also spliut by group here as well

valid_inds, test_inds  = next(GroupShuffleSplit(test_size=.6, n_splits=2, random_state = 7).split(test, groups=test['Group_id']))
valid = test.iloc[valid_inds].drop("Group_id",axis=1)
test = test.iloc[test_inds].drop("Group_id",axis=1)
test = test.loc[test["last"]==1]

print("train",train.shape[0],"valid",valid.shape[0],"test",test.shape[0])

In [ ]:
### filter for most frequent targets

if KEEP_TOP_K_TARGETS > 0 :
    print(df_feat.shape[0])
    df_feat = df_feat.loc[df_feat["city_id"].isin(TOP_TARGETS)]
    print(df_feat.shape[0])    
    assert df_feat["city_id"].nunique() == KEEP_TOP_K_TARGETS

In [ ]:
# ########################
# ## stratified train/test split by class - then drop any train rows with overlap wit htest IDs.  Could also stratify by users, but risks some classes being non present in test
# ### split could maybe be by utrip ID ? 
# ### orig - split by group : 

# # train_inds, test_inds = next(GroupShuffleSplit(test_size=.2, n_splits=2, random_state = 7).split(df_feat, groups=df_feat['user_id']))
# # X_train = df_feat.iloc[train_inds].drop(DROP_FEATS,axis=1,errors="ignore")
# # X_test = df_feat.iloc[test_inds].drop(DROP_FEATS,axis=1,errors="ignore")
# # assert (set(X_train[TARGET_COL].unique()) == set(X_test[TARGET_COL].unique()))
# #################
# ## alt: split by class. May be leaky! 
# X_train, X_test = train_test_split(df_feat,stratify=df_feat[TARGET_COL])

# ########################
# print("X_train",X_train.shape[0])
# ## get last row in trip only in test/eval set: 
# print("X_test",X_test.shape[0])

# ### following is for splitting by group - can't do so currebntly as group/user id col is missing
# # X_test = X_test.loc[X_test["utrip_steps_from_end"]==1] # last row per trip
# # print("X_test after filtering for last instance per trip",X_test.shape[0])

# # y_train = X_train.pop(TARGET_COL)
# # y_test = X_test.pop(TARGET_COL)

# # print("# classes",y_train.nunique())

# # # ## check that same classes in train and test - 
# # # assert (set(y_train.unique()) == set(y_test.unique()))